In [56]:
import pandas as pd
import os
import warnings

warnings.filterwarnings("ignore")

In [ ]:
def clean_columns(columns_list: list[str]) -> list[str]:
    return [column.strip().replace('  ', ' ').replace('  ', ' ').replace('.', '').replace(' ', '_').upper() for column in columns_list]

In [94]:
df_base = pd.read_excel("BASE CORREOS.xlsx")
df_base.columns = clean_columns(df_base.columns)

df_base.rename(columns={
    'OFICINA': 'ID_OFICINA', 
    'CORREOS': 'CORREO',
    'CARGO': 'PUESTO',
    }, inplace=True)

df_base['CORREO'] = df_base['CORREO'].str.upper()

df_base.dropna(inplace=True)

print(df_base.shape)
df_base.head(3)

(332, 7)


,ID_OFICINA,CORREO,TERRITORIO,NOMBRE_OFICINA,PUESTO,REGISTRO,NOMBRE
0,2,"TEYZAGUIRRE@BBVA.COM, DMALAGA@BBVA.COM, MIRKO....",GT CALLAO SAN MIGUEL,OFICINA PANDO,GERENTE DE OFICINA,P015142,"EYZAGUIRRE GONZALES, TATIANA"
1,5,"JVEGAC@BBVA.COM, KGALVEZ@BBVA.COM, MIRKO.IRIAR...",GT NORTE CHICO,OFICINA MEGAPLAZA CHIMBOTE,GERENTE DE OFICINA,P011496,"CASTRO PAZ, GALBA"
2,10,"EVIVAS@BBVA.COM, RICHARD.ORDAYA@BBVA.COM, MIRK...",GT CENTRO,OFICINA ANDAHUAYLAS,GERENTE DE OFICINA,P013079,"VILLEGAS HUAMANI, NELLA"


In [95]:
df_directorio = pd.read_excel("T&C - Directorio BCOM.xlsx")
df_directorio.columns = clean_columns(df_directorio.columns)

df_directorio.rename(columns={
    'CENTRO_DE_COSTO': 'ID_OFICINA',
    'NOMBRE_CENTRO_DE_COSTO': 'NOMBRE_OFICINA',
    }, inplace=True)

cols_req = ['ID_OFICINA', 'NOMBRE_OFICINA', 'REGISTRO', 'NOMBRE', 'CORREO', 'PUESTO']
df_directorio = df_directorio[cols_req]

df_directorio.dropna(inplace=True)

df_directorio = df_directorio[df_directorio['PUESTO'].str.contains('GERENTE', case=False)].reset_index(drop=True)

df_directorio['PUESTO'] = df_directorio['PUESTO'].replace({
    '(?i).*SUB GERENTE.*': 2,
    '(?i).*GERENTE.*': 1
    }, regex=True)

df_directorio.sort_values(by=['ID_OFICINA','PUESTO'], inplace=True, ignore_index=True)
df_directorio.reset_index(drop=True, inplace=True)

print(df_directorio.shape)
df_directorio.head(3)

(562, 6)


,ID_OFICINA,NOMBRE_OFICINA,REGISTRO,NOMBRE,CORREO,PUESTO
0,5,OFICINA MEGAPLAZA CHIMBOTE,P018307,"VEGA CANCINO, JENNIFER",JVEGAC@BBVA.COM,1
1,5,OFICINA MEGAPLAZA CHIMBOTE,P008733,"GALVEZ AGUIRRE, CATHERINE JULIANA",KGALVEZ@BBVA.COM,2
2,10,OFICINA ANDAHUAYLAS,P014618,"VIVAS CUZCANO, EVER ROY",EVIVAS@BBVA.COM,1


In [96]:
def concatenar_correos(grupo: pd.DataFrame) -> pd.Series:
    gerente_correo = grupo.loc[grupo['PUESTO'] == 1, 'CORREO'].values
    subgerente_correo = grupo.loc[grupo['PUESTO'] == 2, 'CORREO'].values
    
    if len(gerente_correo) > 0 and len(subgerente_correo) > 0:
        correos_concatenados = gerente_correo[0] + ', ' + subgerente_correo[0]
    elif len(gerente_correo) > 0:
        correos_concatenados = gerente_correo[0]
    else:
        correos_concatenados = subgerente_correo[0]
    
    return pd.Series({
        'ID_OFICINA': grupo['ID_OFICINA'].iloc[0],
        'PUESTO': 'GERENTE DE OFICINA',
        'CORREO': correos_concatenados,
        'REGISTRO': grupo['REGISTRO'].iloc[0],
        'NOMBRE': grupo['NOMBRE'].iloc[0]
        })

df_directorio_final = df_directorio.groupby('ID_OFICINA').apply(concatenar_correos).reset_index(drop=True)

print(df_directorio_final.shape)
df_directorio_final.head(3)

(288, 5)


,ID_OFICINA,PUESTO,CORREO,REGISTRO,NOMBRE
0,5,GERENTE DE OFICINA,"JVEGAC@BBVA.COM, KGALVEZ@BBVA.COM",P018307,"VEGA CANCINO, JENNIFER"
1,10,GERENTE DE OFICINA,"EVIVAS@BBVA.COM, RICHARD.ORDAYA@BBVA.COM",P014618,"VIVAS CUZCANO, EVER ROY"
2,13,GERENTE DE OFICINA,"MGUTIERREZC@BBVA.COM, JJOFERNANDEZ@BBVA.COM",P008766,"GUTIERREZ CECCARELLI, MANUEL PIEZZINI"


In [105]:
df_actualizado = pd.merge(df_base, df_directorio_final, on='ID_OFICINA', how='left')

df_actualizado['REGISTRO'] = df_actualizado['REGISTRO_y'].fillna(df_actualizado['REGISTRO_x'])
df_actualizado['NOMBRE'] = df_actualizado['NOMBRE_y'].fillna(df_actualizado['NOMBRE_x'])
df_actualizado['PUESTO'] = df_actualizado['PUESTO_y'].fillna(df_actualizado['PUESTO_x'])
df_actualizado['CORREO'] = df_actualizado['CORREO_y'].fillna(df_actualizado['CORREO_x']).str.upper()
df_actualizado['ID_OFICINA'] = df_actualizado['ID_OFICINA'].astype(str).str.zfill(4)

df_actualizado.drop(columns=['REGISTRO_x', 'REGISTRO_y', 'NOMBRE_x', 'NOMBRE_y', 'PUESTO_x', 'PUESTO_y', 'CORREO_x', 'CORREO_y'], inplace=True)

correos_nuevos = ['MIRKO.IRIARTE@BBVA.COM', 'PAULA.ZUNIGA@BBVA.COM']

def agregar_correos(lista_correos: list) -> str:
    for correo in correos_nuevos:
        if correo not in lista_correos:
            lista_correos.append(correo)
    return ', '.join(lista_correos)

df_actualizado['CORREO'] = df_actualizado['CORREO'].str.split(', ').apply(agregar_correos)

cols_req_final = ['ID_OFICINA', 'CORREO', 'TERRITORIO', 'NOMBRE_OFICINA', 'PUESTO', 'REGISTRO', 'NOMBRE']
df_actualizado = df_actualizado[cols_req_final]
print(df_actualizado.shape)
df_actualizado.head(3)

(332, 7)


,ID_OFICINA,CORREO,TERRITORIO,NOMBRE_OFICINA,PUESTO,REGISTRO,NOMBRE
0,0002,"TEYZAGUIRRE@BBVA.COM, DMALAGA@BBVA.COM, MIRKO....",GT CALLAO SAN MIGUEL,OFICINA PANDO,GERENTE DE OFICINA,P015142,"EYZAGUIRRE GONZALES, TATIANA"
1,0005,"JVEGAC@BBVA.COM, KGALVEZ@BBVA.COM, MIRKO.IRIAR...",GT NORTE CHICO,OFICINA MEGAPLAZA CHIMBOTE,GERENTE DE OFICINA,P018307,"VEGA CANCINO, JENNIFER"
2,0010,"EVIVAS@BBVA.COM, RICHARD.ORDAYA@BBVA.COM, MIRK...",GT CENTRO,OFICINA ANDAHUAYLAS,GERENTE DE OFICINA,P014618,"VIVAS CUZCANO, EVER ROY"


In [106]:
df_actualizado.to_excel('CORREOS_ACTUALIZADOS.xlsx', index=False)
os.startfile('CORREOS_ACTUALIZADOS.xlsx')